In [138]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import cv2
import os

In [94]:
path = 'img_rgb.jpg'
path1 = 'file.jpg'
path2 = 'reconstiued222.jpg'
imag = Image.open(path)

In [95]:
def image_details(path):
    # Ouvrir l'image à partir du chemin donné
    img = Image.open(path)
    
    # Obtenir la taille de l'image (largeur, hauteur)
    size = img.size
    
    # Obtenir le format de l'image (JPEG, PNG, etc.)
    format = img.format
    
    # Obtenir le mode de l'image (RGB, L, etc.)
    mode = img.mode
    
    # Obtenir la résolution de l'image si disponible (en DPI)
    resolution = img.info.get('dpi')
    
    # Calculer la définition de l'image (nombre total de pixels)
    definition = size[0] * size[1]
    
    # Stocker également la taille originale de l'image
    original_size = size
    
    # Si la résolution est disponible, ajuster la taille en fonction de la résolution
    if resolution is not None:
        size = (size[0] / resolution[0], size[1] / resolution[1])
    else:
        size = (0, 0)  # ou toute résolution par défaut que vous souhaitez utiliser
    
    # Obtenir la taille du fichier en kilobytes
    file_size_kb = os.path.getsize(path) / 1000.0
    
    # Calculer le nombre total de bits de l'image
    if mode == 'RGB':
        total_bits = definition * 3 * img.bits
    else:
        total_bits = definition * img.bits
    
    # Convertir le nombre total de bits en kilobits
    total_bits_kb = total_bits / (1024 * 8)
    
    # Calculer le taux de compression en pourcentage
    compression_ratio = 100 * (1 - (file_size_kb / total_bits_kb))
    
    # Afficher les informations collectées avec les unités
    print("Dimensions réelles:", size, "inches")
    print("Format:", format)
    print("Mode:", mode)
    print("Resolution:", resolution, "DPI")
    print("Définition 1:", definition, "pixels")
    print("Définition 2:", original_size, "pixels")
    print("Taille en mémoire avec compression:", file_size_kb, "kB")
    print("Taille réelle sans compression:", total_bits_kb/1024, "MB")
    print("Taux de compression: ", compression_ratio, "%")
    
    # Retourner toutes les informations collectées sous forme de tuple
    return size, format, mode, resolution, definition, original_size, file_size_kb, total_bits_kb, compression_ratio

In [96]:
result = image_details('eren_rgb.jpeg')

Dimensions réelles: (4.2, 2.36) inches
Format: JPEG
Mode: RGB
Resolution: (300, 300) DPI
Définition 1: 892080 pixels
Définition 2: (1260, 708) pixels
Taille en mémoire avec compression: 111.596 kB
Taille réelle sans compression: 2.5522613525390625 MB
Taux de compression:  95.73004274653992 %


In [97]:
def plot_images(original_path, modified_path):
    # Ouvrir l'image originale depuis le chemin spécifié
    img1 = Image.open(original_path)
    
    # Ouvrir l'image modifiée (reconstruite) depuis le chemin spécifié
    img2 = Image.open(modified_path)
    
    # Déterminer le type d'image de l'image originale
    img1_mode = img1.mode
    
    # Créer une figure et des sous-graphiques pour afficher les deux images côte à côte
    fig, axs = plt.subplots(1, 2, figsize=(30, 15))  # Créer une figure avec une rangée et deux colonnes
    fig.subplots_adjust(wspace=0.01)  # Ajuster l'espace entre les sous-graphiques
    
    # Afficher l'image originale dans le premier sous-graphique
    if img1_mode == 'L':
        axs[0].imshow(img1, cmap='gray')
        axs[0].set_title('Image initiale (niveau de gris)')  # Définir le titre du premier sous-graphique
    elif img1_mode == '1':
        axs[0].imshow(img1, cmap='binary')
        axs[0].set_title('Image initiale (binaire)')  # Définir le titre du premier sous-graphique
    else:
        axs[0].imshow(img1)
        axs[0].set_title('Image initiale')  # Définir le titre du premier sous-graphique
    axs[0].axis('off')  # Désactiver les axes du premier sous-graphique
    
    # Afficher l'image reconstruite dans le deuxième sous-graphique
    if img1_mode == 'L':
        axs[1].imshow(img2, cmap='gray')
        axs[1].set_title('Image reconstruite (niveau de gris)')  # Définir le titre du deuxième sous-graphique
    elif img1_mode == '1':
        axs[1].imshow(img2, cmap='binary')
        axs[1].set_title('Image reconstruite (binaire)')  # Définir le titre du deuxième sous-graphique
    else:
        axs[1].imshow(img2)
        axs[1].set_title('Image reconstruite')  # Définir le titre du deuxième sous-graphique
    axs[1].axis('off')  # Désactiver les axes du deuxième sous-graphique
    
    # Ajuster la disposition de la figure pour éviter les chevauchements
    fig.tight_layout()
    
    # Afficher la figure avec les deux images
    plt.show()

In [98]:
def rgb_to_ycrcb(img):
    rgb_image = np.array(img)
    ycrcb_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2YCrCb)
    return ycrcb_image

In [99]:
def ycrcb_to_rgb(ycrcb_image):
    ycrcb_image = np.array(ycrcb_image)
    rgb_image = cv2.cvtColor(ycrcb_image, cv2.COLOR_YCrCb2RGB)
    # Convert numpy array to PIL Image
    rgb_image_pil = Image.fromarray(rgb_image)
    # Save the PIL Image
    return rgb_image_pil

In [100]:
def block_subdivision(image, taille_bloc):

    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    # Obtenir la largeur et la hauteur de l'image
    largeur, hauteur = image.size

    # Calculer le nombre de blocs en largeur et en hauteur
    blocs_largeur = (largeur + taille_bloc - 1) // taille_bloc
    blocs_hauteur = (hauteur + taille_bloc - 1) // taille_bloc

    # Créer une nouvelle image avec les dimensions ajustées
    nouvelle_largeur = blocs_largeur * taille_bloc
    nouvelle_hauteur = blocs_hauteur * taille_bloc
    nouvelle_image = Image.new(image.mode, (nouvelle_largeur, nouvelle_hauteur), color='black')
    nouvelle_image.paste(image, (0, 0))

    blocs = []
    # Parcourir chaque bloc
    for y in range(blocs_hauteur):
        for x in range(blocs_largeur):
            # Obtenir les coordonnées du coin supérieur gauche du bloc
            coin_sup_gauche_x = x * taille_bloc
            coin_sup_gauche_y = y * taille_bloc
            # Obtenir les coordonnées du coin inférieur droit du bloc
            coin_inf_droit_x = (x + 1) * taille_bloc
            coin_inf_droit_y = (y + 1) * taille_bloc
            # Extraire le bloc de l'image
            bloc = nouvelle_image.crop((coin_sup_gauche_x, coin_sup_gauche_y, coin_inf_droit_x, coin_inf_droit_y))
            blocs.append(bloc)
    
    return blocs, (largeur, hauteur)

In [101]:
def reconstruct_image(blocs, taille_bloc, dimensions_originales):
    largeur_orig, hauteur_orig = dimensions_originales
    
    # Créer une nouvelle image vide avec les dimensions originales
    image_reconstituee = np.zeros((hauteur_orig, largeur_orig, 3), dtype=np.uint8)
    
    # Parcourir chaque bloc et le placer à la position correspondante dans l'image reconstituée
    idx = 0
    for y in range(0, hauteur_orig, taille_bloc):
        for x in range(0, largeur_orig, taille_bloc):
            # Extraire le bloc de l'image reconstituée
            bloc = blocs[idx]
            idx += 1
            # Convertir le bloc en tableau numpy
            bloc_np = np.array(bloc, dtype=np.uint8)
            
            if bloc_np.shape[2] > bloc_np.shape[0]:
                bloc_np = np.transpose(bloc_np, (2, 1, 0))
            # Calculer les coordonnées de collage du bloc dans l'image reconstituée
            x_end = min(x + taille_bloc, largeur_orig)
            y_end = min(y + taille_bloc, hauteur_orig)
            # Coller le bloc dans l'image reconstituée
            image_reconstituee[y:y_end, x:x_end] = bloc_np[:y_end-y, :x_end-x]

    # Créer une image PIL à partir du tableau numpy
    image_reconstituee_pil = Image.fromarray(image_reconstituee)
    
    return image_reconstituee_pil

In [102]:
def chroma_subsampling(transcol, SSV, SSH):
    # Convert Image to NumPy array
    transcol_np = np.array(transcol)
    
    # Filtrage des canaux de chrominance
    #crf = cv2.boxFilter(transcol_np[:,:,1], ddepth=-1, ksize=(2,2))
    #cbf = cv2.boxFilter(transcol_np[:,:,2], ddepth=-1, ksize=(2,2))
    
    crf = transcol_np[:,:,1]
    cbf = transcol_np[:,:,2]
    
    # Sous-échantillonnage des canaux de chrominance
    crsub = crf[::SSV, ::SSH]
    cbsub = cbf[::SSV, ::SSH]
    
    # Création de la liste contenant tous les canaux
    imSub = [np.array(transcol_np[:,:,0]), np.array(crsub), np.array(cbsub)]
    
    return imSub

In [103]:
def inverse_chroma_subsampling(imSub, SSV, SSH):
    # Récupérer les canaux sous-échantillonnés
    lum, _, _ = imSub
    
    # Initialiser des tableaux pour les canaux complets
    H, W = imSub[1].shape
    crf = np.zeros((H * SSH, W * SSV))
    cbf = np.zeros((H * SSH, W * SSV))
    
    # Créer l'image complète
    image_complete = np.zeros((H * SSV, W * SSH, 3), dtype=np.uint8)
    image_complete[:,:,0] = lum
    image_complete[:,:,1] = np.repeat(np.repeat(imSub[1], SSH, axis=0), SSV, axis=1)
    image_complete[:,:,2] = np.repeat(np.repeat(imSub[2], SSH, axis=0), SSV, axis=1)
    
    # Convertir le tableau numpy en image PIL
    transcol = Image.fromarray(image_complete)
    
    return transcol

In [104]:
def apply_dct(matrix):
    # Appliquer la DCT à chaque matrice
    dct_matrice0 = dct(dct(matrix[0].T, norm='ortho').T, norm='ortho')
    dct_matrice1 = dct(dct(matrix[1].T, norm='ortho').T, norm='ortho')
    dct_matrice2 = dct(dct(matrix[2].T, norm='ortho').T, norm='ortho')

    dct_matrix = [dct_matrice0, dct_matrice1, dct_matrice2]
    
    return dct_matrix

In [105]:
def inverse_dct(dct_matrix):
    # Appliquer l'inverse de la DCT à chaque matrice
    inverse_dct_matrice0 = idct(idct(dct_matrix[0].T, norm='ortho').T, norm='ortho')
    inverse_dct_matrice1 = idct(idct(dct_matrix[1].T, norm='ortho').T, norm='ortho')
    inverse_dct_matrice2 = idct(idct(dct_matrix[2].T, norm='ortho').T, norm='ortho')

    inverse_dct_matrix = [inverse_dct_matrice0, inverse_dct_matrice1, inverse_dct_matrice2]
    
    return inverse_dct_matrix

In [106]:
def reshape_and_average(matrix1):
        # Redimensionne la matrice en une matrice 4x4 en prenant la moyenne de chaque bloc 2x2
        reshaped = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                # Calcule la moyenne des 4 valeurs dans le bloc 2x2
                block_mean = np.mean(matrix1[2*i:2*(i+1), 2*j:2*(j+1)])
                reshaped[i, j] = block_mean
        return reshaped

In [107]:
def quantization(matrix):
    # Récupération des composantes de l'image à partir de la liste
    luminance = matrix[0]  # Composante de luminance (Y)
    chrominance_U = matrix[1]
    chrominance_V = matrix[2]
    
    # Réplication des échantillons pour chaque bloc 2x2
    Cr = np.kron(chrominance_U, np.ones((2, 2)))
    Cb = np.kron(chrominance_V, np.ones((2, 2)))
    
    # Matrices de quantification de luminance et de chrominance
    luminance_quantization_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                              [12, 12, 14, 19, 26, 58, 60, 55],
                                              [14, 13, 16, 24, 40, 57, 69, 56],
                                              [14, 17, 22, 29, 51, 87, 80, 62],
                                              [18, 22, 37, 56, 68, 109, 103, 77],
                                              [24, 35, 55, 64, 81, 104, 113, 92],
                                              [49, 64, 78, 87, 103, 121, 120, 101],
                                              [72, 92, 95, 98, 112, 100, 103, 99]])

    chrominance_quantization_matrix = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                                [18, 21, 26, 66, 99, 99, 99, 99],
                                                [24, 26, 56, 99, 99, 99, 99, 99],
                                                [47, 66, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99]])
    
    # Applique les matrices de quantification à chaque composante
    luminance_quantized = np.round(luminance / luminance_quantization_matrix)
    chrominance_Cr_quantized = np.round(Cr / chrominance_quantization_matrix)
    chrominance_Cb_quantized = np.round(Cb / chrominance_quantization_matrix)
    
    chrominance_Cr_quantized = np.round(reshape_and_average(chrominance_Cr_quantized))
    chrominance_Cb_quantized = np.round(reshape_and_average(chrominance_Cb_quantized))
    
    quantized = [luminance_quantized, chrominance_Cr_quantized, chrominance_Cb_quantized]
    
    return quantized

In [108]:
def dequantization(matrix):
    # Récupération des composantes de l'image à partir de la liste
    L = matrix[0]  # Composante de luminance (Y)
    chrominance_Cr_quantized = matrix[1]
    chrominance_Cb_quantized = matrix[2]
    
    # Réplication des échantillons pour chaque bloc 2x2
    Cr = np.kron(chrominance_Cr_quantized, np.ones((2, 2)))
    Cb = np.kron(chrominance_Cb_quantized, np.ones((2, 2)))

    # Matrices de quantification de luminance et de chrominance
    luminance_quantization_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                                              [12, 12, 14, 19, 26, 58, 60, 55],
                                              [14, 13, 16, 24, 40, 57, 69, 56],
                                              [14, 17, 22, 29, 51, 87, 80, 62],
                                              [18, 22, 37, 56, 68, 109, 103, 77],
                                              [24, 35, 55, 64, 81, 104, 113, 92],
                                              [49, 64, 78, 87, 103, 121, 120, 101],
                                              [72, 92, 95, 98, 112, 100, 103, 99]])

    chrominance_quantization_matrix = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                                                [18, 21, 26, 66, 99, 99, 99, 99],
                                                [24, 26, 56, 99, 99, 99, 99, 99],
                                                [47, 66, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99],
                                                [99, 99, 99, 99, 99, 99, 99, 99]])

    # Multiplie chaque valeur quantifiée par sa correspondance dans la matrice de quantification respective
    luminance_dequantized = L * luminance_quantization_matrix
    chrominance_Cr_dequantized = Cr * chrominance_quantization_matrix
    chrominance_Cb_dequantized = Cb * chrominance_quantization_matrix

    # Restaure la résolution de chrominance U et V à 8x8
    chrominance_Cr_dequantized = reshape_and_average(chrominance_Cr_dequantized)
    chrominance_Cb_dequantized = reshape_and_average(chrominance_Cb_dequantized)

    # Retourne la liste des composantes [luminance, chrominance_U, chrominance_V]
    dequantized = [luminance_dequantized, chrominance_Cr_dequantized, chrominance_Cb_dequantized]

    return dequantized


In [109]:
def zigzag_scan(matrix):
    Y = np.array(matrix[0], dtype=np.float64)
    Cr = np.array(matrix[1], dtype=np.float64)
    Cb = np.array(matrix[2], dtype=np.float64)

    Y = np.concatenate([np.diagonal(Y[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - Y.shape[0], Y.shape[0])])
    Cr = np.concatenate([np.diagonal(Cr[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - Cr.shape[0], Cr.shape[0])])
    Cb = np.concatenate([np.diagonal(Cb[::-1,:], i)[::(2*(i % 2)-1)] for i in range(1 - Cb.shape[0], Cb.shape[0])])
    
    ycrcb = np.concatenate((Y, Cr, Cb))
    return ycrcb.tolist()

In [110]:
def inverse_zigzag_scan(ycrcb):
    def inverse_vecteur_zigzag(vector, shape):
        # Initialiser une matrice avec des valeurs nulles
        rows, cols = shape
        mat = np.array([[None] * cols for _ in range(rows)], dtype=np.float64)
        row, col = 0, 0
        direction = 1

        # Remplir la matrice en suivant le modèle de balayage en zigzag
        for i in range(rows * cols):
            mat[row][col] = vector[i]
            if direction == 1:
                if col == cols - 1:
                    row += 1
                    direction = -1
                elif row == 0:
                    col += 1
                    direction = -1
                else:
                    row -= 1
                    col += 1
            else:
                if row == rows - 1:
                    col += 1
                    direction = 1
                elif col == 0:
                    row += 1
                    direction = 1
                else:
                    row += 1
                    col -= 1

        return mat
    
    def diviser_liste(liste):
        # Vérifier si la liste a au moins 96 éléments
        if len(liste) < 96:
            return "La liste doit contenir au moins 95 éléments pour être divisée selon les spécifications."
        
        # Diviser la liste en trois parties
        premiere_partie = liste[:64]
        deuxieme_partie = liste[64:80]
        troisieme_partie = liste[80:]

        return premiere_partie, deuxieme_partie, troisieme_partie
    
    # Séparer les composantes Y, Cr et Cb
    Y, Cr, Cb = diviser_liste(ycrcb)
    
    # Appliquer la fonction inverse de balayage en zigzag à chaque composante
    Y = inverse_vecteur_zigzag(Y, (8, 8))
    Cr = inverse_vecteur_zigzag(Cr, (4, 4))
    Cb = inverse_vecteur_zigzag(Cb, (4, 4))
    
    # Arrondir les valeurs
    Y = np.round(Y)
    Cr = np.round(Cr)
    Cb = np.round(Cb)
    
    return [Y, Cr, Cb]


In [111]:
def rle_encode(data):
    encoded_data = ""
    current_char = data[0]
    count = 1

    for i in range(1, len(data)):
        if data[i] == current_char:
            count += 1
        else:
            if count > 1:  # Ajout de cette condition
                encoded_data += str(current_char) + "_" + str(count) + "*"
            else:
                encoded_data += str(current_char) + "*"
            current_char = data[i]
            count = 1

    if count > 1:  # Ajout de cette condition
        encoded_data += str(current_char) + "_" + str(count)
    else:
        encoded_data += str(current_char)

    return encoded_data

In [112]:
def rle_decode(encoded_data):
    decoded_data = []
    encoded_data_split = encoded_data.split('*')

    for item in encoded_data_split:
        if "_" in item:
            char, count = item.split("_")
            char = float(char)
            if char.is_integer():
                char = int(char)
            decoded_data.extend([char] * int(count))
        else:
            char = float(item)
            if char.is_integer():
                char = int(char)
            decoded_data.append(char)

    return decoded_data

In [113]:
def ext_chr(chaine):
    occ = {c: chaine.count(c) for c in set(chaine)}
    return sorted(occ.items(), key=lambda x: x[1], reverse=True)

# Fonction principale implémentant le codage de Huffman
def huffman_dictionnary_code(node, binString=''):
    if isinstance(node, str):
        return {node: binString}
    (l, r) = node
    d = {}
    d.update(huffman_dictionnary_code(l, binString + '0'))
    d.update(huffman_dictionnary_code(r, binString + '1'))
    return d

# Construction de l'arbre de Huffman
def huffman_tree(chaine):
    nodes = ext_chr(chaine)
    
    while len(nodes) > 1:
        (key1, c1) = nodes.pop()
        (key2, c2) = nodes.pop()
        node = (key1, key2)
        nodes.append((node, c1 + c2))
        nodes.sort(key=lambda x: x[1], reverse=True)
        
    return nodes[0][0]

In [114]:
def encoding_huffman(string):
    nodes = huffman_tree(string)
    huffmanCode = huffman_dictionnary_code(nodes)
    
    compressed_string = ''
    for char in string:
        compressed_string += huffmanCode[char]
    return compressed_string, nodes

In [115]:
def decoding_huffman(compressed_string, huffman_tree):
    decoded_string = ''
    current_node = huffman_tree
    
    for bit in compressed_string:
        if bit == '0':
            current_node = current_node[0]
        else:
            current_node = current_node[1]
        
        if isinstance(current_node, str):
            decoded_string += current_node
            current_node = huffman_tree
    
    return decoded_string

In [116]:
def diviser_longue_chaine(chaine, tailles_sous_chaines):
    sous_chaines = []
    index_debut = 0
    for taille in tailles_sous_chaines:
        sous_chaine = chaine[index_debut:index_debut+taille]
        sous_chaines.append(sous_chaine)
        index_debut += taille
    return sous_chaines

In [117]:
def concatenate_huffman_codes(strings):
  compressed_string = ''
  huffman_trees = []
  huffman_code_lengths = []
  
  for code in strings:
    compressed_string += encoding_huffman(code)[0]
    huffman_trees.append(encoding_huffman(code)[1])
    huffman_code_lengths.append(len(encoding_huffman(code)[0]))

  return compressed_string, huffman_trees, huffman_code_lengths

In [118]:
def deconcatenate_huffman_codes(compressed_string, huffman_trees, huffman_code_lengths):
    # Divise la chaîne compressée en sous-chaînes
    liste = diviser_longue_chaine(compressed_string, huffman_code_lengths)
    decoded = []
    
    for i in range(len(liste)):
        # Décode chaque sous-chaîne en utilisant l'arbre Huffman correspondant
        elt = decoding_huffman(liste[i], huffman_trees[i])
        decoded.append(elt)
    
    return decoded

In [119]:
# Fonction pour la compression intermédiaire JPEG
def intermediate_jpeg_compression(image):
    # Chaîne de compression
    compressed_data = ''
    
    # Conversion de l'image RGB en espace de couleur YCrCb
    ycrcb_code = rgb_to_ycrcb(image)
    
    # Sous-division de l'image en blocs de 8x8 et récupération de la forme originale
    sub_blocks, original_shape = block_subdivision(ycrcb_code, 8)
    
    # Liste pour stocker les codes compressés de chaque bloc
    compressed_blocks = []
    
    # Pour chaque bloc de sous-division
    for block in sub_blocks:
        # Sous-échantillonnage des composantes Chroma (Cr, Cb) par un facteur de 2x2
        sub_sampled_block = chroma_subsampling(block, 2, 2)
        
        # Application de la transformée en cosinus discrète (DCT) au bloc
        dct_transformed_block = apply_dct(sub_sampled_block)
        
        # Quantification des coefficients DCT
        quantized_block = quantization(dct_transformed_block)
        
        # Balayage zigzag des coefficients quantifiés
        zigzag_scanned_block = zigzag_scan(quantized_block)
        
        # Codage RLE des coefficients balayés en zigzag
        rle_encoded_block = rle_encode(zigzag_scanned_block)
        
        # Ajout du code compressé du bloc à la liste
        compressed_blocks.append(rle_encoded_block)
    
    # Concaténation des codes Huffman des blocs
    compressed_data, huffman_trees, code_lengths = concatenate_huffman_codes(compressed_blocks)
    
    return compressed_data, huffman_trees, code_lengths, original_shape

In [120]:
# Fonction pour la décompression intermédiaire JPEG
def intermediate_jpeg_decompression(compressed_data, huffman_trees, code_lengths, original_shape):
    # Déconcaténation des codes Huffman
    huffman_decoded_blocks = deconcatenate_huffman_codes(compressed_data, huffman_trees, code_lengths)
    
    # Liste pour stocker les blocs décompressés
    decompressed_blocks = []
    
    # Pour chaque bloc décodé
    for block in huffman_decoded_blocks:
        # Décodage RLE
        rle_decoded_block = rle_decode(block)
        
        # Reconstruction du balayage zigzag
        zigzag_scanned_block = inverse_zigzag_scan(rle_decoded_block)
        
        # Déquantification des coefficients
        dequantized_block = dequantization(zigzag_scanned_block)
        
        # Transformation inverse de la DCT
        inverse_dct_block = inverse_dct(dequantized_block)
        
        # Reconstruction du sous-échantillonnage
        inverse_subsampled_block = inverse_chroma_subsampling(inverse_dct_block, 2, 2)
        
        # Ajout du bloc décompressé à la liste
        decompressed_blocks.append(inverse_subsampled_block)
    
    # Reconstitution de l'image à partir des blocs décompressés
    reconstructed_image = reconstruct_image(decompressed_blocks, 8, original_shape)
    
    # Conversion de l'espace de couleur YCrCb en RGB
    rgb_image = ycrcb_to_rgb(reconstructed_image)
    
    return rgb_image

In [150]:
def create_color_palette(image, num_colors):
    image = np.array(image)
    # Convertir l'image en un tableau 2D de pixels
    pixels = np.reshape(image, (-1, 3))  # (nombre de pixels, 3 canaux de couleur)

    # Appliquer l'algorithme de k-means clustering
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixels)

    # Obtenir les centres des clusters (couleurs dominantes)
    color_palette = kmeans.cluster_centers_.astype(int)

    # Associer chaque pixel à l'indice de couleur dans la palette
    labels = kmeans.predict(pixels)

    # Reformater les indices des couleurs dans la palette selon la forme de l'image originale
    palette_indices = np.reshape(labels, image.shape[:2])

    return color_palette, palette_indices

In [151]:
def inverse_color_palette(color_palette, palette_indices):
    # Récupérer les dimensions de l'image à partir des indices de palette
    height, width = palette_indices.shape

    # Initialiser une image vide avec les dimensions récupérées
    reconstructed_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Remplir l'image reconstruite avec les couleurs de la palette
    for i in range(height):
        for j in range(width):
            color_index = palette_indices[i, j]
            reconstructed_image[i, j] = color_palette[color_index]

    return reconstructed_image

In [161]:
def map_to_palette(image, color_palette):
    image = np.array(image)
    pixels = np.reshape(image, (-1, 3))  # (nombre de pixels, 3 canaux de couleur)

    # Calculer la distance de chaque pixel à chaque couleur de la palette
    distances = np.linalg.norm(pixels[:, np.newaxis] - color_palette, axis=2)

    # Obtenir l'indice de la couleur la plus proche pour chaque pixel
    indices = np.argmin(distances, axis=1)

    # Reformater les indices des couleurs selon la forme de l'image originale
    mapped_indices = np.reshape(indices, image.shape[:2])

    return mapped_indices.flatten(), np.array(image).shape

In [162]:
def inverse_map_to_palette(mapped_indices, color_palette, image_shape):
    # Convertir les indices mappés en tableau NumPy si ce n'est pas déjà le cas
    mapped_indices = np.asarray(mapped_indices)

    # Réorganiser les couleurs de la palette en fonction des indices mappés
    image_colors = color_palette[mapped_indices]

    # Reformater les couleurs en une image selon la forme donnée
    reconstructed_image = np.reshape(image_colors, image_shape)

    return reconstructed_image, color_palette

In [163]:
def fusionner2(liste):
    tailles = []
    nombres_fusionnes = ""
    
    # Parcourir tous les éléments de la liste
    for nombre in liste:
        # Convertir chaque nombre en chaîne de caractères et l'ajouter à la chaîne fusionnée
        nombres_fusionnes += str(nombre)
        
        # Ajouter la taille de l'élément à la liste des tailles
        tailles.append(len(str(nombre)))
    
    # Retourner la chaîne de caractères fusionnée et la liste des tailles
    return nombres_fusionnes, tailles

In [164]:
def separer2(chaine, tailles):
    nombres = []
    debut = 0
    
    # Parcourir toutes les tailles dans la liste
    for taille in tailles:
        # Extraire le sous-chaîne correspondant à la taille actuelle
        sous_chaine = chaine[debut:debut+taille]
        
        # Convertir la sous-chaîne en nombre et l'ajouter à la liste des nombres
        nombres.append(int(sous_chaine))
        
        # Mettre à jour la position de départ pour la prochaine sous-chaîne
        debut += taille
    
    # Retourner la liste des nombres
    return nombres

In [165]:
def encoding_lzw(data, color_palette):
    alphabet = [str(i) for i in range(len(color_palette))]
    data_fused, taille = fusionner2(data)  # Fusionner les chaînes de caractères
    encoded_data = []

    dictionary = {}  # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[char] = i

    prefix = ''
    for char in data_fused:
        new_entry = prefix + char
        if new_entry in dictionary:
            prefix = new_entry
        else:
            encoded_data.append(dictionary[prefix])
            dictionary[new_entry] = len(dictionary)
            prefix = char

    if prefix:
        encoded_data.append(dictionary[prefix])

    return encoded_data, taille

In [166]:
def decoding_lzw(compressed_data, color_palette, tailleL):
    alphabet = [str(i) for i in range(len(color_palette))]
    result = []
    dictionary = {}
    current_code = len(alphabet)

    # Initialiser le dictionnaire avec les caractères de l'alphabet spécifié
    for i, char in enumerate(alphabet):
        dictionary[i] = char

    old_entry = dictionary[compressed_data[0]]
    result.append(old_entry)
    for new_entry in compressed_data[1:]:
        if new_entry in dictionary:
            entry = dictionary[new_entry]
        elif new_entry == current_code:
            entry = old_entry + old_entry[0]
        else:
            raise ValueError("Mauvaise séquence compressée")

        result.append(entry)

        # Utilisez le même dictionnaire pour la décompression
        dictionary[current_code] = old_entry + entry[0]
        current_code += 1
        old_entry = entry

    result = ''.join(result)
    result = separer2(result, tailleL)
    
    return result

In [167]:
def intermediaire_compression_gif(image, num_colors):
    color_palette, palette_indices = create_color_palette(image, num_colors)
    mapper, shape = map_to_palette(image, color_palette)
    compressed_data, taille = encoding_lzw(mapper, color_palette)
    
    return compressed_data, color_palette, palette_indices, shape, taille

In [168]:
def intermediaire_decompression_gif(compressed_data, color_palette, palette_indices, shape, taille):
    # Décompression LZW
    code1 = decoding_lzw(compressed_data, color_palette, taille)
    _, color_palette1 = inverse_map_to_palette(code1, color_palette, shape)
    code3 = inverse_color_palette(color_palette1, palette_indices)
    image = Image.fromarray(code3)  # Convertir le tableau NumPy en objet Image PIL
    return image